In [13]:
import warnings
warnings.simplefilter(action="ignore", category=UserWarning)

import pandas as pd
import psycopg

db = psycopg.connect("dbname=next host=localhost port=5432 user=postgres password=postgres")

factions = pd.read_sql_query("select * from open_discourse.factions", db)
print(factions)

    id  abbreviation                                          full_name
0   -1     not found                                          not found
1    0           AfD                        Alternative für Deutschland
2    1           BHE      Block der Heimatvertriebenen und Entrechteten
3    2            BP                                       Bayernpartei
4    3         Grüne                              Bündnis 90/Die Grünen
5    4       CDU/CSU  Christlich Demokratische Union Deutschlands/Ch...
6    5            DA                  Demokratische Arbeitsgemeinschaft
7    6    DIE LINKE.                                         DIE LINKE.
8    7            DP                                    Deutsche Partei
9    8        DP/DBP             Deutsche Partei/Deutsche Partei Bayern
10   9        DP/FVP                  Deutsche Partei/Freie Volkspartei
11  10           DPB                             Deutsche Partei Bayern
12  11           DRP                              Deutsche Refor

In [14]:
speeches = pd.read_sql_query("select * from open_discourse.speeches limit 10", db)
print(speeches)

   id  session  electoral_term first_name last_name  politician_id  \
0   0        2               1               Köhler       11001150   
1   1        2               1               Arnold             -1   
2   2        2               1               Köhler       11001150   
3   3        2               1                Heuss             -1   
4   4        2               1               Köhler       11001150   
5   5        2               1                Heuss             -1   
6   6        3               1               köhler       11001150   
7   7        3               1             adenauer       11000009   
8   8        3               1               köhler       11001150   
9   9        4               1               köhler       11001150   

                                      speech_content  faction_id  \
0  Meine Damen und Herren! Ich eröffne die 2. Sit...          -1   
1    Der Bundesrat ist versammelt, Herr Präsident.\n          -1   
2  Ich danke für diese Er

In [43]:
speeches = pd.read_sql_query("""
    SELECT speech_content, factions.abbreviation, electoral_term, date
    FROM open_discourse.speeches
    JOIN open_discourse.factions ON factions.id = speeches.faction_id
    WHERE faction_id <> -1
    """, db)
print(speeches.head())

                                      speech_content abbreviation  \
0  Herr Präsident! Meine Damen und Herren! Ich ha...      CDU/CSU   
1  Herr Kollege, darf ich aus Ihren Bemerkungen e...          SPD   
2  Herr Kollege Jacobi, ich habe nur auf Nachrich...      CDU/CSU   
3  Herr Präsident! Meine Damen und Herren! Bei de...      CDU/CSU   
4  Herr Präsident! Meine sehr verehrten Damen und...          SPD   

   electoral_term        date  
0               6  1970-01-28  
1               6  1970-01-28  
2               6  1970-01-28  
3               6  1970-01-28  
4               6  1970-01-28  


In [19]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("deutsche-telekom/gbert-large-paraphrase-euclidean")

In [46]:
speeches = speeches.head(10) # TEMP!
encodings = model.encode(speeches["speech_content"].values)
print(encodings.shape)
print(encodings[:10])

(10, 1024)
[[-7.0756847e-01 -3.4419537e-01  4.3384778e-01 ... -1.4995003e-01
  -9.1015011e-01 -3.4411293e-01]
 [-1.2585453e+00 -1.7530520e-01  7.9750115e-01 ...  2.5267097e-01
   5.2476808e-02 -3.5706678e-01]
 [ 4.2741127e-02  1.3332956e-01  3.8915557e-01 ...  5.6434262e-01
  -1.7314014e-01 -4.3241632e-01]
 ...
 [-9.9570984e-01 -1.0827211e+00  5.9563708e-01 ...  1.1401446e+00
   2.1521583e-01 -4.9163952e-01]
 [ 4.3479707e-02  1.1258184e+00  8.5270800e-02 ...  2.0066404e+00
   2.0930728e-01 -5.5554384e-01]
 [-5.0648755e-01  1.7351361e-02  4.2048320e-02 ...  1.9620563e-01
   6.5841695e-04 -8.2075417e-02]]
